# DR7&MPL8

In [35]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [48]:


import cross_match as cm
from astropy.io import fits
from astropy.cosmology import WMAP9 as cosmo
from collections import OrderedDict

#############################################################################
dir_drp = '/Users/astro/Documents/notebooks/manga/spectro/analysis/MPL-8/drpall-v2_5_3.fits'
dir_dr7 = '/Users/astro/Documents/notebooks/zs/data/gal_info_dr7_v5_2.fit'
dir_k_10_not_enough = '/Users/astro/Documents/notebooks/zs/cross_match/cross_match_res/v4/cmatch_10_not_enough.txt'
dir_res = '/Users/astro/Documents/notebooks/zs/cross_match/cross_match_res/v4/'
############################################################################

def cross():
    
    with fits.open(dir_drp) as f_drp, fits.open(dir_dr7) as f_dr7:

        data = f_drp[1].data
        manga = np.empty((len(data), 2), dtype=np.float64)
        manga[:, 0] = data['objra']
        manga[:, 1] = data['objdec']
        manga_z = data['z']
        plateifu = data['plateifu']

        data1 = f_dr7[1].data
        dr7 = np.empty((len(data1), 2), dtype=np.float64)
        dr7[:, 0] = data1['RA']
        dr7[:, 1] = data1['DEC']
        dr7_z = data1['Z']

        i=np.where(plateifu=='8158-3703')[0][0]
        print(i,manga[i])
        cm.cmatch(plateifu[i],manga[i][0],manga[i][1],manga_z[i],dr7,dr7_z,dir_res,dir_k_10_not_enough)


if __name__ == '__main__':
        
    cross()

1519 [62.55180236 -4.98373809]
8158-3703


ValueError: zero-size array to reduction operation maximum which has no identity

# cross_match_395_k>10

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from collections import OrderedDict

import import_ipynb
import cross_match
from astropy.cosmology import WMAP9 as cosmo
from astropy.io import fits
from astropy.table import Table



#############################################################################
dir_drp = '/Users/astro/Documents/notebooks/manga/spectro/analysis/MPL-8/drpall-v2_5_3.fits'
dir_dr7 = '/Users/astro/Documents/notebooks/zs/data/gal_info_dr7_v5_2.fit'
dir_k_10_not_enough = '/Users/astro/Documents/notebooks/zs/cross_match/cross_match_res/v2/cmatch_10_not_enough.txt'
dir_res = '/Users/astro/Documents/notebooks/zs/cross_match/cross_match_res/v2/'
############################################################################


def cross():

    with fits.open(dir_drp) as f_drp, fits.open(dir_dr7) as f_dr7:

        f_10 = Table.read(dir_k_10_not_enough, format='ascii')
        manga = np.empty((len(f_10), 2), dtype=np.float64)
        manga[:, 0] = f_10['col2']
        manga[:, 1] = f_10['col3']
        plateifu = f_10['col1']

        data = f_drp[1].data
        Z = data['Z']
        pifu = data['plateifu']

        data1 = f_dr7[1].data
        dr7 = np.empty((len(data1), 2), dtype=np.float64)
        dr7[:, 0] = data1['RA']
        dr7[:, 1] = data1['DEC']
        dr7_z = data1['Z']

        for i in range(0, len(manga)):

            index = np.where(pifu == plateifu[i])[0]
            z = Z[index]
            dis = cosmo.comoving_distance(z)  # Mpc
            r = 0.15 / dis.value
            max_r = cross_match.r2arcsec(r)  # degree

            dist, ind = cross_match.crossmatch_angular(manga[i], dr7,20, max_r)
            match = ~np.isinf(dist)
            dist_match = dist[match]
            ind_match = ind[match]

            ind_1 = list(OrderedDict.fromkeys(list(ind_match)))
            match_1 = []  # dist index which remove multiple elements
            for k in ind_1:
                index = np.where(ind_match == k)[0][0]
                match_1.append(dist_match[index])

            if len(ind_match) == len(ind_1) and len(ind_1) > 0:
                if len(dist[0]) == len(dist_match):
                    print('20 is not enough', plateifu[i], manga[:, 0][i],
                          manga[:, 1][i], max_r)
                elif len(dist[0]) > len(dist_match):
                    with open(dir_res + plateifu[i] + '.txt', 'a+') as f_res:
                        print('index',
                              'ra',
                              'dec',
                              'min_dis_deg',
                              'max_r',
                              file=f_res)
                        for j in range(0, len(ind_1)):
                            ra = dr7[:, 0][ind_1[j]]
                            dec = dr7[:, 1][ind_1[j]]
                            print(ind_1[j],
                                  ra,
                                  dec,
                                  match_1[j],
                                  max_r,
                                  file=f_res)

if __name__ == '__main__':

    cross()

importing Jupyter notebook from cross_match.ipynb
20 is not enough 10001-12704 133.989966869 57.6779676669 [0.14115479]
20 is not enough 10507-1902 176.525541667 47.4820555556 [0.65845641]
20 is not enough 10509-12703 168.545376088 48.318515932 [0.27914476]
20 is not enough 10510-12704 179.117267145 55.1252090259 [0.53556613]
20 is not enough 10517-12702 149.99409 5.32659 [1.97975657]
20 is not enough 7958-6101 257.383688857 34.4270318778 [0.08361879]
20 is not enough 8078-3703 42.3874325272 -0.784442553068 [0.0842864]
20 is not enough 8079-1901 42.407980379 -0.813857447855 [0.08140749]
20 is not enough 8079-6101 43.1562184278 -1.16906729877 [0.08903433]
20 is not enough 8080-12702 47.9994154757 -1.16166126075 [0.07460794]
20 is not enough 8080-12705 47.8890844551 -0.742939934572 [0.0708262]
20 is not enough 8080-1901 47.9851588295 -0.0325628998893 [0.05324287]
20 is not enough 8080-3701 47.528362637 -0.468033748749 [0.05456005]
20 is not enough 8080-3704 49.4574544521 -0.554658522356 

20 is not enough 9877-1901 195.070939008 28.0639222565 [0.09749924]
20 is not enough 9877-3704 194.662274497 27.9539019989 [0.09985051]
20 is not enough 9877-6104 195.090290656 27.8985596385 [0.12265471]
20 is not enough 9880-1901 195.078678476 28.0093027853 [0.09651556]
20 is not enough 9880-3701 194.86968999 28.0405183648 [0.10638562]


In [2]:
from astropy.table import Table

t=Table.read('/Users/astro/Documents/notebooks/zs/cross_match/cross_match_res/v2/cmatch_10_not_enough.txt',format='ascii')
print(t,len(t))

    col1         col2          col3             col4        
----------- ------------- ------------- --------------------
10001-12704 133.989966869 57.6779676669  0.14115478769125356
 10001-3701 132.465646765 57.1437279024 0.050848013169786446
 10001-3702 132.912768243 57.1074235568  0.07803981454301591
 10001-6102 132.653991697 57.3596675436  0.07655848771578087
 10213-3703      123.8433      44.06446  0.09834188009019654
 10219-1901  125.14376637 21.1054663161   0.1182713927146769
 10219-1902 125.062375384 21.0600550691  0.11999492813396592
 10219-6102 125.148697904 21.0677945565   0.1325546782566985
10221-12703     124.05695        25.076  0.12770682318244184
10221-12705 124.109635921 23.1425158913  0.14425187053577215
        ...           ...           ...                  ...
  9877-1902 195.270098514 27.8917529314  0.10310391900048708
  9877-3704 194.662274497 27.9539019989  0.09985051496205223
  9877-6104 195.090290656 27.8985596385  0.12265470861052226
  9879-1901 197.34533081